In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Add Other Todos and start with the intial setup

- [ ] Add data preprocessing
- [ ] Train initial model
- [ ] Evaluate results